In [1]:
import geopandas as gpd
import folium
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# Read Train data
RailStations = gpd.read_file('../data/cleaned/RailStationsMerged.geojson')
RailStations.head()

,Name,Description,StationType,StationName,StationCode,StationLine,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,MRT,Ang Mo Kio,NS16,North-South,"POLYGON Z ((103.84988 1.36925 0, 103.84976 1.3..."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,MRT,Buangkok,NE15,North-East,"POLYGON Z ((103.89304 1.38166 0, 103.89283 1.3..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,LRT,Bakau,SE3,Sengkang LRT,"POLYGON Z ((103.90538 1.38786 0, 103.90529 1.3..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,LRT,Riviera,PE4,Punggol LRT,"POLYGON Z ((103.916 1.39444 0, 103.91634 1.394..."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,LRT,Fernvale,SW5,Sengkang LRT,"POLYGON Z ((103.8765 1.39148 0, 103.87648 1.39..."


In [3]:
# Read Bus data
BusStops = gpd.read_file('../data/cleaned/BusStops.geojson')
BusStops.head()

,BUS_STOP_N,BUS_ROOF_N,LOC_DESC,geometry
0,65059,B12,ST ANNE'S CH,POINT (103.9013 1.39303)
1,16171,B06,YUSOF ISHAK HSE,POINT (103.77437 1.29892)
2,61101,NIL,BLK 120,POINT (103.8637 1.33564)
3,1239,B01,SULTAN PLAZA,POINT (103.86165 1.30285)
4,17269,B01,BLK 730,POINT (103.76264 1.30492)


In [4]:
# First we need to reproject both GeoDataFrames
rail_stations = RailStations.to_crs(epsg=3857)
bus_stops = BusStops.to_crs(epsg=3857)

# Initialize an empty list to hold the nearest bus stop and distance for each station
nearest_bus_stops_output = []

# Initialize an empty list to hold the nearest 10 bus stops and distances for each station
nearest_bus_stops_output = []

# Iterate through each rail station using tqdm to show a progress bar
for i, row in tqdm(rail_stations.iterrows(), total=rail_stations.shape[0]):

    # Compute the distance from each rail station to all bus stops
    distances = bus_stops.distance(row['geometry'])

    # Sort the distances and get the top 10 nearest bus stops
    nearest_10 = distances.nsmallest(10)
    
    # Append the nearest 10 bus stops and distances to the output list
    nearest_bus_stops_output.append({
        'StationName': row['StationName'],
        'Nearest Bus Stops': bus_stops.loc[nearest_10.index, "BUS_STOP_N"].tolist(),  # List of nearest bus stops
        'Distances (m)': nearest_10.tolist()  # List of distances to the nearest bus stops
    })

# Convert the output into a DataFrame for easier visualization
nearest_10_df = pd.DataFrame(nearest_bus_stops_output)

# Output the nearest 10 bus stops for each station
nearest_10_df.head(10)  # Display first 10 rows


  0%|          | 0/253 [00:00<?, ?it/s]

100%|██████████| 253/253 [00:02<00:00, 104.13it/s]


,StationName,Nearest Bus Stops,Distances (m)
0,Ang Mo Kio,"[54009, 54261, 54399, 54269, 54391, 54339, 543...","[72.83868005584391, 84.49303002463226, 102.629..."
1,Buangkok,"[67601, 67609, 67711, 67461, 67629, 67621, 673...","[0.0, 18.43958406406945, 248.17477823258378, 2..."
2,Bakau,"[67131, 67539, 67531, 67121, 67129, 67139, 675...","[31.247173992762843, 106.87109159074058, 123.9..."
3,Riviera,"[65231, 65239, 65181, 65269, 65261, 65189, 651...","[21.106476528376874, 22.10845310643349, 250.57..."
4,Fernvale,"[67631, 67639, 67489, 67481, 67491, 67499, 675...","[35.9173118925268, 43.12872973868515, 74.68994..."
5,Punggol,"[65009, 65251, 65259, 65359, 65219, 65351, 653...","[16.17538122301639, 42.36925898638381, 54.0867..."
6,Punggol,"[65009, 65251, 65259, 65359, 65219, 65351, 653...","[16.17538122301639, 42.36925898638381, 54.0867..."
7,Bendemeer,"[60019, 60011, 7359, 7361, 7351, 60099, 7369, ...","[0.0, 0.0, 160.69798418699304, 193.24638149793..."
8,Kupang,"[67551, 67559, 67701, 67561, 67569, 67741, 675...","[268.3340230842661, 304.32701747323904, 414.71..."
9,Jalan Besar,"[7529, 7419, 7589, 1109, 7539, 7531, 7551, 112...","[45.571210840822815, 64.1166232550563, 156.639..."


In [6]:
from folium.plugins import MarkerCluster
from ipywidgets import interact
import ipywidgets as widgets

In [14]:

# Function to find bus stops within a specified radius (500 meters)
def find_bus_stops_within_radius(station_name, radius_meters, rail_stations_gdf, bus_stops_gdf):
    # Get the geometry for the given station
    station = rail_stations_gdf[rail_stations_gdf['StationName'].str.lower() == station_name.lower()]
    
    if station.empty:
        return f"Station '{station_name}' not found."
    
    station_geom = station.geometry.iloc[0]

    # Compute the distance from the station to all bus stops
    distances = bus_stops_gdf.distance(station_geom)

    # Filter bus stops within the specified radius
    nearby_bus_stops = bus_stops_gdf[distances <= radius_meters]

    return nearby_bus_stops

# Reset to geographic CRS for plotting in Folium (EPSG:4326)
rail_stations = rail_stations.to_crs(epsg=4326)
bus_stops = bus_stops.to_crs(epsg=4326)

# Function to plot the train station and its nearest bus stops on a Folium map
def plot_station_with_bus_stops(station_name):
    # Create a base map centered at Singapore
    singapore_center = [1.3521, 103.8198]  # Coordinates for Singapore
    m = folium.Map(location=singapore_center, zoom_start=12)

    # Find the nearest bus stops for the selected station
    nearest_bus_stops = find_bus_stops_within_radius(station_name, 500, rail_stations, bus_stops)

    # Get the selected train station's details
    selected_station = rail_stations[rail_stations['StationName'].str.lower() == station_name.lower()]

     # Print the nearest bus stops details
    if not nearest_bus_stops.empty:
        print(f"Nearest Bus Stops for Station: {station_name}")
        print(nearest_bus_stops[['BUS_STOP_N', 'LOC_DESC', 'geometry']])
        print("-" * 50)  # Separator for readability
    
    # Plot the selected train station on the map
    if not selected_station.empty:
        # Check if the geometry is a Point or Polygon, and use the centroid for Polygons
        station_geom = selected_station.geometry.iloc[0]
        if station_geom.geom_type == 'Point':
            station_coords = [station_geom.y, station_geom.x]
        elif station_geom.geom_type == 'Polygon':
            station_coords = [station_geom.centroid.y, station_geom.centroid.x]

        folium.Marker(
            location=station_coords,
            popup=f"{station_name} (Train Station)",
            icon=folium.Icon(color='red', icon='train')
        ).add_to(m)

    # Plot nearest bus stops
    if not nearest_bus_stops.empty:
        marker_cluster = MarkerCluster().add_to(m)
        for idx, bus_stop in nearest_bus_stops.iterrows():
            bus_stop_coords = [bus_stop.geometry.y, bus_stop.geometry.x]
            bus_stop_info = f"Bus Stop: {bus_stop['BUS_STOP_N']}<br>Location: {bus_stop['LOC_DESC']}"
            folium.Marker(
                location=bus_stop_coords,
                popup=bus_stop_info,
                icon=folium.Icon(color='blue', icon='bus')
            ).add_to(marker_cluster)

    # Display the map
    return m

# Create an interactive dropdown to select the train station
station_dropdown = widgets.Dropdown(
    options=rail_stations['StationName'].unique(),
    description='Train Station:',
    disabled=False
)

# Use IPyWidgets to make the map interactive
interact(lambda station_name: plot_station_with_bus_stops(station_name), station_name=station_dropdown)

interactive(children=(Dropdown(description='Train Station:', options=('Ang Mo Kio', 'Buangkok', 'Bakau', 'Rivi…

<function __main__.<lambda>(station_name)>